In [1]:
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
qb = QuantBook()

Add data for selected financial stocks

In [ ]:
tickers = ["JPM", "BAC", "MS", "SCHW", "GS", "AXP", "C"] # Financials
symbols = [qb.AddEquity(ticker, Resolution.Daily).Symbol for ticker in tickers]

Get Price to Earnings Ratio data

In [ ]:
pe_ratios = qb.GetFundamental(symbols, 
                              "ValuationRatios.PERatio", 
                              datetime(2021, 1, 1),
                              datetime(2022, 1, 1))
pe_ratios.head()

Rename columns for better readability

In [ ]:
pe_ratios.columns = ["American Express", "JPMorgan", "Goldman Sachs", "Morgan Stanley", "Bank of America", "Schwab", "Citigroup"]

Plot PE-Ratio over time

In [ ]:
pe_ratios.plot(figsize=(16, 8), title="PE Ratio Over Time")
plt.xlabel("Time")
plt.ylabel("PE Ratio")
plt.show()

Compare average PE-Ratio

In [ ]:
mean_pe = pe_ratios.mean()
mean_pe.sort_values()

Add price data for the symbols

In [ ]:
history = qb.History(symbols, 
                     datetime(2021, 1, 1),
                     datetime(2022, 1, 1),
                     Resolution.Daily).close.unstack(level=0)
history.columns = ["American Express", "JPMorgan", "Goldman Sachs", "Morgan Stanley", "Bank of America", "Schwab", "Citigroup"]
history.head()

Plot returns over time

In [ ]:
returns_over_time = ((history.pct_change()[1:] + 1).cumprod() - 1)
returns_over_time.plot(figsize=(16, 8), title="Returns Over Time")
plt.grid()
plt.ylabel("Return")
plt.show()

Correlation Matrix

In [ ]:
np.corrcoef(returns_over_time.tail(1), mean_pe)

Scatter Plot

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(returns_over_time.tail(1), mean_pe)
plt.title("2021 Returns / PERatio")
plt.xlabel("2021 Returns")
plt.ylabel("Mean PERatio 2021")
plt.grid()
plt.show()

### Add Options Data

In [ ]:
bac = qb.AddOption("BAC")
bac.SetFilter(-5, 5, timedelta(20), timedelta(50))

In [ ]:
option_history = qb.GetOptionHistory(bac.Symbol, datetime(2021, 1, 1), datetime(2021, 1, 10))
print (option_history.GetStrikes())
print (option_history.GetExpiryDates())
option_history.GetAllData()

### Add Indicators

In [ ]:
# Example with BB, it is a datapoint indicator
# Define the indicator
bb = BollingerBands(30, 2) # 30 day MA +/- 2 std deviation

# Gets historical data of indicator
bbdf = qb.Indicator(bb, "BAC", 360, Resolution.Daily, Field.Open)

bbdf.plot(figsize=(16, 8), title="BAC Bollinger Bands", grid=1)

In [ ]:
# drop undesired fields
bbdf = bbdf.drop(["standarddeviation", "percentb", "bandwidth"], 1)

In [ ]:
bbdf.plot(figsize=(16, 8), title="BAC Bollinger Bands", grid=1)

#### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
history = qb.History(qb.Symbol("BAC"), 60, Resolution.Daily)

In [ ]:
# Drop the symbol index from multi-index dataframe
history = history.reset_index(level=0, drop=True)

# Get list of 60 days of open prices from dataframe
prices = list(history['close'])

we use bollinger bands middle band (SMA based on open price) as input to predict closing price

In [ ]:
# Format time series data into form compatible with LinearRegression
train_X = np.asarray(bbdf["middleband"][-60:-30]).reshape(30,1)

# First 30 days of price data used for training
train_Y = prices[:30]

# Define LinearRegression model from scikit
reg = LinearRegression()

# Fit model with training data
reg.fit(train_X, train_Y)

In [ ]:
# Format time series data
test_X = np.asarray(bbdf["middleband"][-60:]).reshape(60,1)

# Use linear regression model to predict prices
prices_pred = reg.predict(test_X)

In [ ]:
plt.figure(figsize=(10,7))

# The first 30 days of price data used to train model
plt.scatter(bbdf["middleband"][-60:-30], prices[:30], color='blue')

# The next 30 days of price data
plt.scatter(bbdf["middleband"][-30:], prices[30:60], color="green")

# The linear regression model
plt.plot(bbdf["middleband"][-60:], prices_pred, color='red', linewidth=1)
plt.xlabel("Middleband")
plt.ylabel("Closing Price")
plt.show()